# Train Model with DPO

### imports

In [1]:
!pip install trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 366.3/366.3 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 29.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [1]:
from datasets import load_dataset
from trl import DPOConfig, DPOTrainer
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import torch

In [3]:
# from google.colab import drive
# drive.mount('/content/drive')

### Login huggingface and load data

In [4]:
! huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: fineGrained).
The token `topicmodeling` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `topi

In [2]:
dataset = load_dataset("EliasHossain/youtube-titles-dpo")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


### load model

We are gonna use the unsloth library which is faster at fine-tuning job. Before that we need to connect with T4 GPU from the runtime menu.

In [6]:
! pip install unsloth

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.1/47.1 kB 2.2 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.6/275.6 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 101.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.8/145.8 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.5/31.5 MB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 865.2/865.2 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.5/156.5 MB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.1/393.1 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 87.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7

Install the required library to use the unsloth

In [1]:
%%capture
# Installs Unsloth, Xformers (Flash Attention) and all other packages!
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

# We have to check which Torch version for Xformers (2.3 -> 0.0.27)
from torch import __version__; from packaging.version import Version as V
xformers = "xformers==0.0.27" if V(__version__) < V("2.4.0") else "xformers"
!pip install --no-deps {xformers} trl peft accelerate bitsandbytes triton

In [3]:
from unsloth import FastLanguageModel
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported
from datasets import load_dataset, Dataset
import pandas as pd
import json, yaml
import torch

<ipython-input-3-f5768de860b0>:1: UserWarning: WARNING: Unsloth should be imported before trl, transformers, peft to ensure all optimizations are applied. Your code may run slower or encounter memory issues without these optimizations.

Please restructure your imports with 'import unsloth' at the top of your file.
  from unsloth import FastLanguageModel


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [4]:
from unsloth import FastLanguageModel
import torch

# Set model name
model_name = "unsloth/Qwen3-14B-unsloth-bnb-4bit"  # Use this exact quantized model

# Load model and tokenizer with Unsloth
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_name,
    max_seq_length = 2048,       # You can increase this if needed
    load_in_4bit = True,         # Efficient memory usage
    load_in_8bit = False,        # Keep False if using 4bit
    full_finetuning = False,     # Set True only if you're planning full finetuning
    # token = "hf_...",          # Only needed for gated models
)

# Ensure pad_token is set properly
tokenizer.pad_token = tokenizer.eos_token


==((====))==  Unsloth 2025.5.10: Fast Qwen3 patching. Transformers: 4.52.4.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.3.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.30. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

We now add LoRA adapters so we only need to update 1 to 10% of all parameters!

In [5]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 32,           # Choose any number > 0! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 32,  # Best to choose alpha = rank or rank*2
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,   # We support rank stabilized LoRA
    loftq_config = None,  # And LoftQ
)

Unsloth 2025.5.10 patched 40 layers with 40 QKV layers, 40 O layers and 40 MLP layers.


### generate title with base model

In [6]:
def format_chat_prompt(user_input, system_message="You are a helpful assistant."):
    """
    Formats user input into the chat template format with <|im_start|> and <|im_end|> tags.

    Args:
        user_input (str): The input text from the user.

    Returns:
        str: Formatted prompt for the model.
    """

    # Format user message
    user_prompt = f"<|im_start|>user\n{user_input}<|im_end|>\n"

    # Start assistant's turn
    assistant_prompt = "<|im_start|>assistant\n"

    # Combine prompts
    formatted_prompt = user_prompt + assistant_prompt

    return formatted_prompt

In [7]:
from transformers import pipeline

# Set up text generation pipeline (DO NOT set device manually)
generator = pipeline("text-generation", model=model, tokenizer=tokenizer)

# Example prompt — update this based on your actual structure
prompt = format_chat_prompt(dataset['valid']['prompt'][0][0]['content'])

# Generate output
outputs = generator(prompt, max_length=100, truncation=True, num_return_sequences=1, temperature=0.7)

# Print the result
print(outputs[0]['generated_text'])

Device set to use cuda:0


<|im_start|>user
Given the YouTube video idea write an engaging title.

**Video Idea**: intro independent component analysis

**Additional Guidance**:
- Title should be between 30 and 75 characters long
- Only return the title idea, nothing else!<|im_end|>
<|im_start|>assistant
<think>
Okay, the user wants a YouTube video title about introducing independent component analysis. Let me start by understanding what ICA is. It's a technique used in signal processing and machine learning to separate mixed signals into


### train model

In [8]:
# ft_model_name = model_name.split('/')[1].replace("Instruct", "DPO")

# training_args = DPOConfig(
#     output_dir=ft_model_name,
#     logging_steps=25,
#     per_device_train_batch_size=8,
#     per_device_eval_batch_size=8,
#     num_train_epochs=3,
#     load_best_model_at_end=True,
#     metric_for_best_model="eval_loss",
#     save_strategy="epoch",
#     eval_strategy="epoch",
#     eval_steps=1,
# )

# device = torch.device('mps')

from trl import DPOTrainer, DPOConfig  # or the appropriate trainer if you're using custom code
import torch

# Adjust fine-tuned model name
ft_model_name = model_name.split('/')[1].replace("Instruct", "DPO")

# Define DPO training configuration
training_args = DPOConfig(
    output_dir=ft_model_name,
    logging_steps=25,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    save_strategy="epoch",
    eval_strategy="epoch",
    eval_steps=1,
)



In [15]:
# # Take a small subset to prevent session crash
# train_subset = dataset['train'].select(range(10))   # Take first 100 samples
# eval_subset = dataset['valid'].select(range(20))     # Take first 20 samples

In [16]:
# tokenizer.chat_template = "{% for message in messages %}{{ message['content'] }} {% endfor %}"

In [13]:
print(dataset["train"].column_names)

['prompt', 'chosen', 'rejected']


In [14]:
# def formatting_func(example):
#     return {"text": f"{example['instruction']}\n{example['response']}"}

# # Preprocess dataset BEFORE passing to trainer
# formatted_train = dataset["train"].map(formatting_func)
# formatted_valid = dataset["valid"].map(formatting_func)


In [15]:
from trl import DPOTrainer, DPOConfig

# Define DPO training arguments
training_args = DPOConfig(
    output_dir="qwen3-dpo-checkpoint",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    learning_rate=5e-5,
    lr_scheduler_type="linear",
    warmup_ratio=0.1,
    logging_steps=10,
    save_strategy="epoch",
    eval_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    seed=42,
    report_to="none"
)

# Initialize DPOTrainer
trainer = DPOTrainer(
    model=model,
    tokenizer=tokenizer,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["valid"],
)

# Start training
trainer.train()


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 1,026 | Num Epochs = 3 | Total steps = 387
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 2
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 2 x 1) = 8
 "-____-"     Trainable parameters = 128,450,560/14,000,000,000 (0.92% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Epoch,Training Loss,Validation Loss,rewards / chosen,rewards / rejected,rewards / accuracies,rewards / margins,logps / chosen,logps / rejected,logits / chosen,logits / rejected,eval_logits / chosen,eval_logits / rejected,nll_loss,aux_loss
1,0.587500,0.538106,0.281831,-0.670660,0.698276,0.952491,-76.851631,-91.474724,-2.361934,-2.365531,0,0,0,0
2,0.428800,0.519478,-0.285808,-1.803312,0.698276,1.517504,-82.528008,-102.801254,-2.611689,-2.636275,No Log,No Log,No Log,No Log


Epoch,Training Loss,Validation Loss,rewards / chosen,rewards / rejected,rewards / accuracies,rewards / margins,logps / chosen,logps / rejected,logits / chosen,logits / rejected,eval_logits / chosen,eval_logits / rejected,nll_loss,aux_loss
1,0.587500,0.538106,0.281831,-0.670660,0.698276,0.952491,-76.851631,-91.474724,-2.361934,-2.365531,0,0,0,0
2,0.428800,0.519478,-0.285808,-1.803312,0.698276,1.517504,-82.528008,-102.801254,-2.611689,-2.636275,No Log,No Log,No Log,No Log
3,0.302400,0.523650,0.269042,-1.672285,0.750000,1.941327,-76.979515,-101.490990,-2.504110,-2.544434,No Log,No Log,No Log,No Log


TrainOutput(global_step=387, training_loss=0.4188279015149257, metrics={'train_runtime': 5611.3343, 'train_samples_per_second': 0.549, 'train_steps_per_second': 0.069, 'total_flos': 0.0, 'train_loss': 0.4188279015149257, 'epoch': 3.0})

### use fine-tuned model

In [16]:
# Load the fine-tuned model
ft_model = trainer.model

In [17]:
# Set up text generation pipeline (DO NOT set device manually)
generator = pipeline("text-generation", model=ft_model, tokenizer=tokenizer)

# Example prompt — update this based on your actual structure
prompt = format_chat_prompt(dataset['valid']['prompt'][0][0]['content'])

# Generate output
outputs = generator(prompt, max_length=100, truncation=True, num_return_sequences=1, temperature=0.7)

# Print the result
print(outputs[0]['generated_text'])

Device set to use cuda:0


<|im_start|>user
Given the YouTube video idea write an engaging title.

**Video Idea**: intro independent component analysis

**Additional Guidance**:
- Title should be between 30 and 75 characters long
- Only return the title idea, nothing else!<|im_end|>
<|im_start|>assistant
<think>
Okay, the user wants a YouTube video title about introducing independent component analysis. Let me start by understanding what ICA is. It's a technique used in signal processing and machine learning for separating mixed signals into


### push to HF hub

In [18]:
model_id = f"EliasHossain/{ft_model_name}"
trainer.push_to_hub(model_id)

adapter_model.safetensors:   0%|          | 0.00/514M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

training_args.bin:   0%|          | 0.00/6.67k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/EliasHossain/qwen3-dpo-checkpoint/commit/b8f588af893d68d478cdeb09aef59781f71bf280', commit_message='EliasHossain/Qwen3-14B-unsloth-bnb-4bit', commit_description='', oid='b8f588af893d68d478cdeb09aef59781f71bf280', pr_url=None, repo_url=RepoUrl('https://huggingface.co/EliasHossain/qwen3-dpo-checkpoint', endpoint='https://huggingface.co', repo_type='model', repo_id='EliasHossain/qwen3-dpo-checkpoint'), pr_revision=None, pr_num=None)

In [19]:
format_chat_prompt(dataset['valid']['prompt'][0][0]['content'])

'<|im_start|>user\nGiven the YouTube video idea write an engaging title.\n\n**Video Idea**: intro independent component analysis\n\n**Additional Guidance**:\n- Title should be between 30 and 75 characters long\n- Only return the title idea, nothing else!<|im_end|>\n<|im_start|>assistant\n'